Criando uma tool de busca de temperatura

In [18]:
import requests
import datetime

from langchain.tools import StructuredTool
from langchain.pydantic_v1 import BaseModel, Field


class GetTemperaturaAtualArgs(BaseModel):
    latitude: float = Field(..., description="Latitude do local a ser consultado")
    longitude: float = Field(..., description="Longitude do local a ser consultado")

def get_temperatura_atual(latitude: float, longitude: float):
    URL = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1
    }

    response = requests.get(URL, params=params)

    if response.status_code == 200:
        resultado = response.json()
        
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado["hourly"]["time"]]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))
        
        temp_atual = resultado["hourly"]["temperature_2m"][index_mais_prox]
        
        return temp_atual
    else:
        raise Exception(f"Erro na requisição: {response.status_code} - {response.text}")
    
    
tool_get_temperatura_atual = StructuredTool.from_function(
    func=get_temperatura_atual,
    name="Tool Get Temperatura Atual",
    description="Retorna a temperatura atual em um local",
    args_schema=GetTemperaturaAtualArgs,
    return_direct=True,
)

tool_get_temperatura_atual.run({"latitude": 52.52, "longitude": 13.41})

10.1

In [17]:
tool_get_temperatura_atual

StructuredTool(name='Tool Get Temperatura Atual', description='Tool Get Temperatura Atual(latitude: float, longitude: float) - Retorna a temperatura atual em um local', args_schema=<class '__main__.GetTemperaturaAtualArgs'>, return_direct=True, func=<function get_temperatura_atual at 0x12a751440>)